## Sentencebert를 활용해 문서 키워드 추출하기

* Sentence Bert를 활용해 도서 목차, 도서 소개, 추천사 등의 도서 정보를 하나의 sentnece embedding으로 표현한 뒤

* 도서 정보에서 빈번하게 나오는 단어를 순서대로 추려 word embedding으로 표현하고

* 최종적으로 sentence embedding과 word embedding 간 Cosine Similarity를 계산하여 

* 문서 키워드를 추출함.

### 모델 불러오기

In [1]:
from transformers import ElectraModel, ElectraTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity
from model import SentenceBert
from konlpy.tag import Hannanum
import pandas as pd
import numpy as np


model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")

Sbert = SentenceBert(model, tokenizer)


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Data 불러오기


In [3]:
import pandas as pd
raw_data = pd.read_parquet("../../App/db/data/book_scraping.parquet")

raw_data.head()


,isbn13,title,toc,intro,publisher
0,9791163034254,깡샘의 안드로이드 앱 프로그래밍 with 코틀린,"[첫째마당 안드로이드 앱 개발 준비하기, 개발 환경 준비하기, 안드로이드 스튜디오 ...","[안드로이드 코틀린 분야 위 도서였던 개정판에 이어 개정 판이 출간되었다, 이번 판...","[이 책의 특징 안드로이드 티라미수을 기준으로 내용 및 소스를 업데이트했습니다, 전..."
1,9788966263677,CPython 파헤치기,"[소스 코드에 포함된 것들, 장 개발 환경 구성하기, 편집기와 통합 개발 환경, 비...",[파이썬이 인터프리터 레벨에서 작동하는 방식을 이해하면 파이썬의 기능을 최대한 활용...,[]
2,9791140702473,부모님을 위한 컴퓨터 무작정 따라하기,"[첫째 마당 이것만은 꼭 컴퓨터 기초 지식 다지기, 컴퓨터 자기소개, 컴퓨터 기본 ...",[컴퓨터 앞에 서면 막막해진다는 부모님을 위해 시작한 욜디 의 컴퓨터 기초 강의 이...,[세상 쉬운부모님을 위한 컴퓨터 무작정 따라하기더이상 물어볼 필요 없어요 일상에 힘...
3,9791192932057,챗GPT,"[AI는 이미 당신보다 똑똑하다, 너무 똑똑한 AI의 출현 위기인가 기회인가, 고도...",[출시된 지 얼마 되지도 않아 세상을 뒤흔든 챗GPT는 지금까지 나온 모든 인공지능...,[]
4,9791140702787,쉽게 시작하는 쿠버네티스,"[장 쿠버네티스의 등장, 컨테이너 환경으로의 진화, 쿠버네티스를 학습하기 전에 알아...",[],[모든 것은 기본에서 시작한다 가볍지만 알차게 배우는 쿠버네티스 쿠버네티스는 컨테이...


In [3]:
def tokenize_text(text, max_length=128):
    token = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_length,
        stride=20,
        return_overflowing_tokens=True,
        return_tensors="pt",
    )
    token.pop("overflow_to_sample_mapping")
    return token


### 문서 키워드 추출

* 1단계. max_length개 토큰 이상으로 구성된 문서인 경우 여러 문단으로 나눔.

* 2단계. 문서 내 한글 및 영문 키워드 추출
* 3단계. Sbert를 활용해 문서 및 키워드 embedding
* 4단계. 문서와 키워드 간 cosine similarity 측정
* 5단계. 유사도 높은 단어 순으로 키워드 제공

In [4]:
import utils 

def keyword_extraction_from_doc(
    doc: str, sbert, model,min_num=3, num_rank=20, max_length=128, noun_extractor=Hannanum()
):
    """
    *-- arguments --*

    doc : 문서 정보
    sbert : sbert 모델
    min_num : 키워드 최소 출현 횟수 ex) 문서 내 3회 이상 사용된 경우 추출
    min_rank : 키워드 순위 설정
    max_length : 문장 Tokenizing 범위
    noun_extractor : 한글 명사 추출을 위해 Konlpy 활용

    """

    # Doc embedding
    # Doc Token이 128개 이상인 경우 여러 문장으로 구분
    # ex) 1280개 토큰이 있는 Doc인 경우 128개 토큰이 있는 문장 10개로 구분
    token = tokenize_text(doc, max_length=max_length)

    ### 도서 정보를 나타내는 Sentence Embedding 생성(10개라면 10개 생성)
    logits = sbert(**token)["sentence_embedding"]

    ### Keyword 후보 추출 추출

    # 한글 키워드 추출
    if type(doc) == str:
        han_nouns = noun_extractor.nouns(doc)
    else:
        raise TypeError("doc must be str type.")

    candidates_kor = pd.DataFrame(han_nouns)[0].value_counts()
    candidate_kor_words = candidates_kor[candidates_kor >= min_num].index.values.tolist()
    candidate_kor_words = [i for i in candidate_kor_words if len(i) > 1]

    # 영문 키워드 추출
    book_info_eng = utils.find_eng(doc, min_num=0)

    if book_info_eng:
        candidates_eng = pd.DataFrame(book_info_eng)[0].value_counts()
        candidates_eng_words = candidates_eng[candidates_eng >= min_num].index.values.tolist()
    else:
        candidates_eng_words = []

    # 키워드 총합
    candidate_words = candidate_kor_words + candidates_eng_words

    if candidate_words == False:
        return pd.DataFrame(columns=["유사도"])

    # 키워드에 대한 embedding 구하기
    token_embedding = tokenizing_function(candidate_words)
    last_hidden_state = model(**token_embedding)["last_hidden_state"]

    ### 문서 embedding
    # [CLS], [SEP] 제거 ([CLS], [SEP]을 제거하면 정확도가 올라감.)
    attention_mask = token_embedding["attention_mask"]
    for i in range(attention_mask.size(0)):
        # x = attention mask 1에 포함 된 마지막 index
        x = (attention_mask[i] == 1).nonzero(as_tuple=True)[0][-1]
        attention_mask[i][0] = 0  # [CLS] = 0
        attention_mask[i][x] = 0  # [SEP] = 0

    # 토큰 내 padding 찾기 = [batch_size, src_token, embed_size]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

    # padding인 경우 0 아닌 경우 1을 곱함 = [batch_size, embed_size]
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)

    # 평균을 위한 token 개수 확대
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)

    # Mean Pooling
    result = sum_embeddings / sum_mask

    ### Keyword와 Doc 문장 비교
    sentence_comparsion = []
    len_sentences = logits.size(0)

    # sentence embedding별 키워드 비교
    for i in range(len_sentences):
        sentence_comparsion.append(
            cosine_similarity(logits[i].unsqueeze(0).detach(), result.detach())
        )

    # Max Pooling
    # 하나의 문단이 여러개로 나눠지게 되면 나눠진 개수 만큼 문장과 단어 간 cosine_similiarty를 계산함.
    # 만약 1개의 문단이 10개의 문단으로 나뉘면 단어 하나 당 10개의 cosine_smiliarity가 존재하는 것임. 
    # 10개의 cosine_similarity 중 max값을 채택하겠다는 의미임.
    result = np.max(sentence_comparsion, axis=0)  # Max

    # Ranking 정리
    result = pd.DataFrame(result.T, index=candidate_words, columns=["유사도"]).sort_values(
        by="유사도", ascending=False
    )[:num_rank]

    return result


from random import randrange

# 도서 Random Sampling
num = randrange(len(raw_data))
num = 3 
book_info: str = utils.merge_series_to_str(raw_data.iloc[num])

# 영단어를 한글로 변환 ex) Python => 파이썬
englist = pd.read_csv("../data/preprocess/englist.csv")
book_info_trans = utils.trans_eng_to_han(book_info, englist=englist)


keyword_extraction_from_doc(" ".join(book_info_trans), Sbert, model,num_rank=15)


변환한 도서정보 :  그림과 실습으로 배우는 도커 & 쿠버네티스


,유사도
메타데이터,0.942622
virtualbox,0.922685
쿠버네티스,0.919135
리눅스,0.901917
mysql,0.900849
프롬프트,0.898217
레드마,0.895093
리눅스용,0.894459
디플로이먼트,0.893091
macos,0.890312
